In [ ]:
def frobenius_norm(matrix):
    return np.linalg.norm(matrix, 'fro')

import numpy as np
import time
import matplotlib.pyplot as plt
W = np.array([[1.0000, 0.0372, 0.0100, -0.0219, -0.8478],
                  [0.0372, 1.0000, -0.5449, -0.3757, -0.4849],
                  [0.0100, -0.5449, 1.0000, -0.0381, 0.0996],
                  [-0.0219, -0.3757, -0.0381, 1.0000, 0.4292],
                  [-0.8478, -0.4849, 0.0996, 0.4292, 1.0000]]) #Calculated using nearcorr func in MATLAB

A = np.array([[1.0000, 0, 0, 0, -0.9360],
              [0, 1.0000, -0.5500, -0.3645, -0.5300],
              [0, -0.5500, 1.0000, -0.0351, 0.0875],
              [0, -0.3645, -0.0351, 1.0000, 0.4557],
              [-0.9360, -0.5300, 0.0875, 0.4557, 1.0000]]) #Calculatd using nearcorr func in MATLAB

def create_random_symmetric_matrix(size):
    A = np.random.rand(size, size)
    sym_matrix = (A + A.T) / 2  # Make it symmetric
    return sym_matrix

def measure_time_for_matrices(calculate_func):
    dimensions = list(range(3, 11))
    time_taken = []

    for dimension in dimensions:
        matrix = create_random_symmetric_matrix(dimension)
        start_time = time.time()
        nearest_corr_matrix = calculate_func(matrix)
        end_time = time.time()
        time_taken.append(end_time - start_time)

    # Plot the results
    plt.plot(dimensions, time_taken, marker='o')
    plt.title('Time taken to calculate nearest correlation matrix')
    plt.xlabel('Matrix Dimension')
    plt.ylabel('Time (seconds)')
    plt.grid(True)
    plt.show()

In [ ]:
import numpy as np

def alternating_projections(A, max_iterations=100, tol=1e-6):
    # Ensure the input matrix is symmetric
    A = 0.5 * (A + A.T)

    n = A.shape[0]
    X = A.copy()

    for iteration in range(max_iterations):
        # Projection onto the space of correlation matrices
        X = (X + X.T) / 2
        X = np.maximum(X, 0)
        np.fill_diagonal(X, 1)

        # Projection onto the space of positive semidefinite matrices
        eigenvalues, eigenvectors = np.linalg.eigh(X)
        eigenvalues = np.maximum(eigenvalues, 0)
        X = eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T

        # Check for convergence
        diff = np.linalg.norm(X - A, 'fro')
        if diff < tol:
            break

    return X
start = time.time()
X = alternating_projections(A)
end=time.time()
print()
print(X)
print(frobenius_norm(W-X))


[[1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 5.47085607e-16
  8.75000000e-02]
 [0.00000000e+00 0.00000000e+00 5.29598178e-16 1.00000000e+00
  4.55700000e-01]
 [0.00000000e+00 0.00000000e+00 8.75000000e-02 4.55700000e-01
  1.00000000e+00]]
1.671061381278378


In [ ]:
import numpy as np
from scipy.linalg import svd, norm

def project_positive(C):
    # Project to space of positive semidefinite matrices
    eigenvalues, eigenvectors = np.linalg.eigh(C)
    return eigenvectors.dot(np.diag(np.maximum(eigenvalues, 0))).dot(eigenvectors.T)

def project_symmetric(C):
    # Project to space of symmetric matrices
    return (C + C.T) / 2

def normalize_diagonal(C):
    # Normalize the diagonal of a matrix to 1.
    d = np.sqrt(np.diagonal(C))
    return C / d / d[:, np.newaxis]

def anderson(A, M=5, N=100):
    """
    Compute nearest correlation matrix using alternating projections with Anderson's acceleration.
    """
    # Initialize
    C = A.copy()
    H = []
    for k in range(N):
        # Projection onto symmetric matrices
        C = project_symmetric(C)

        # Projection onto positive semidefinite matrices
        C = project_positive(C)

        # Normalize diagonal and Update history buffer
        C = normalize_diagonal(C)
        H.append(C)

        # Ensure maximum history length
        if len(H) > M:
            H.pop(0)

        if k >= 3:
            # Compute Anderson acceleration update using history buffer
            Δ = H[-1] - H[-2]
            C += Δ
    return C
start = time.time()
X = anderson(A, M=5, N=100)
end=time.time()
print(end-start)
print(X)
print(frobenius_norm(W-X))

0.004732370376586914
[[ 1.          0.02318371  0.00703045 -0.0128971  -0.83930213]
 [ 0.02318371  1.         -0.54219948 -0.36859685 -0.4832012 ]
 [ 0.00703045 -0.54219948  1.         -0.03718672  0.09319802]
 [-0.0128971  -0.36859685 -0.03718672  1.          0.42581655]
 [-0.83930213 -0.4832012   0.09319802  0.42581655  1.        ]]
0.030738943436009362


In [ ]:
import numpy as np
from scipy.linalg import eigh
from numpy.linalg import norm

def nearest_correlation_matrix_higham(A, N, tol):
    # Initialize C as a copy of A
    C = np.copy(A)

    for k in range(1, N + 1):
        # Compute the spectral decomposition of C
        w, Q = eigh(C)
        # Clamp the eigenvalues to the interval [0, 1]
        w = np.maximum(0, np.minimum(w, 1))
        # Reconstruct C as C = QΛQ^T
        C = Q @ np.diag(w) @ Q.T

        # Check for convergence
        if norm(C - A, 'fro') < tol:
            break

        # Normalize the diagonal of C to be all ones
        C[np.diag_indices(C.shape[0])] = 1.0

    return C

N = 100  # Maximum number of iterations
tol = 1e-6  # Tolerance for convergence
start=time.time()
X = nearest_correlation_matrix_higham(A, N, tol)
end=time.time()
print(end-start)
print(X)
print(frobenius_norm(W-X))


0.018413543701171875
[[ 1.00000000e+00 -1.20255773e-11 -1.20254257e-11  1.20256148e-11
  -1.20255599e-11]
 [-1.20255218e-11  1.00000000e+00 -1.20247245e-11  1.20252373e-11
  -1.20250236e-11]
 [-1.20254257e-11 -1.20246967e-11  1.00000000e+00  1.20255084e-11
  -1.20254789e-11]
 [ 1.20255871e-11  1.20252373e-11  1.20255084e-11  1.00000000e+00
   1.20253593e-11]
 [-1.20255321e-11 -1.20249958e-11 -1.20254789e-11  1.20253593e-11
   1.00000000e+00]]
1.7829790856645122


In [ ]:
import numpy as np
from scipy.linalg import eigh
from numpy.linalg import norm, solve

def nearest_correlation_matrix_newton(A, N, tol):
    # Initialize C as a copy of A
    C = np.copy(A)

    for k in range(1, N + 1):
        # Calculate the gradient of the objective function ∇f(C)
        gradient = compute_gradient(C, A)

        # Calculate the Hessian matrix of the objective function ∇^2f(C)
        hessian = compute_hessian(C)

        # Compute the Newton update step ∆C by solving the linear system
        delta_C = solve(hessian, -gradient)

        # Update C: C ← C + ∆C
        C += delta_C

        # Project C onto the space of symmetric matrices
        C = (C + C.T) / 2

        # Project C onto the space of positive semidefinite matrices
        w, Q = eigh(C)
        w = np.maximum(w, 0)
        C = Q @ np.diag(w) @ Q.T

        # Normalize the diagonal of C to be all ones
        C[np.diag_indices(C.shape[0])] = 1.0

        # Check for convergence
        if norm(delta_C, 'fro') < tol:
            break

    return C

def compute_gradient(C, A):
    # Calculate the gradient of the objective function ∇f(C)
    return 2 * (C - A)

def compute_hessian(C):
    # Calculate the Hessian matrix of the objective function ∇^2f(C)
    n = C.shape[0]
    return np.eye(n)


N = 100  # Maximum number of iterations
tol = 1e-6  # Tolerance for convergence

start=time.time()
X = nearest_correlation_matrix_newton(A, N, tol)
end=time.time()
print(end-start)
print(X)
print(frobenius_norm(W-X))


0.007047891616821289
[[ 1.          0.02383632  0.00718581 -0.01320068 -0.88380848]
 [ 0.02383632  1.         -0.54605753 -0.3717425  -0.50136533]
 [ 0.00718581 -0.54605753  1.         -0.03728336  0.09613235]
 [-0.01320068 -0.3717425  -0.03728336  1.          0.43984197]
 [-0.88380848 -0.50136533  0.09613235  0.43984197  1.        ]]
0.06281477855303876


In [ ]:
import numpy as np
from scipy.linalg import eigh

def nearest_correlation_matrix_log_barrier(A, N, mu, rho):
    # Initialize C as a copy of A
    C = np.copy(A)
    dim = A.shape[0]

    for k in range(1, N + 1):
        # Compute the spectral decomposition of C
        w, Q = eigh(C)

        # Update Λ using the barrier function
        w = barrier_function(w, mu)

        # Reconstruct C as C = QΛQ^T
        C = Q @ np.diag(w) @ Q.T

        # Update the barrier parameter
        mu *= rho

        # Normalize the diagonal of C to be all ones
        C[np.diag_indices(dim)] = 1.0

    return C

def barrier_function(w, mu):
    # Update eigenvalues using the barrier function
    return (w + np.sqrt(w**2 + 4 * mu)) / 2


N = 100  # Maximum number of iterations
mu = 1.0  # Initial barrier parameter
rho = 0.9  # Barrier parameter update factor
start = time.time()
X = nearest_correlation_matrix_log_barrier(A, N, mu, rho)
end=time.time()
print(end-start)
print(X)
print(frobenius_norm(W-X))


0.009701728820800781
[[ 1.00000000e+00  1.51248774e-03 -4.59714727e-04 -1.32073670e-03
  -5.25433510e-03]
 [ 1.51248774e-03  1.00000000e+00 -3.27002252e-03 -2.80010477e-03
  -3.77418473e-03]
 [-4.59714727e-04 -3.27002252e-03  1.00000000e+00  6.04641204e-04
   1.37243919e-03]
 [-1.32073670e-03 -2.80010477e-03  6.04641204e-04  1.00000000e+00
   3.26921580e-03]
 [-5.25433510e-03 -3.77418473e-03  1.37243919e-03  3.26921580e-03
   1.00000000e+00]]
1.770961805377326


In [ ]:
import numpy as np
from scipy.linalg import eigh
from numpy.linalg import norm

def nearest_correlation_matrix_diagonal_scaling(A, N, tol):
    # Initialize C as a copy of A
    C = np.copy(A)
    dim = A.shape[0]

    for k in range(1, N + 1):
        # Compute the diagonal scaling matrix D
        D = np.diag(1 / np.sqrt(np.diag(C)))

        # Scale C with D
        C = D @ C @ D

        # Project C onto the space of positive semidefinite matrices
        w, U = eigh(C)
        w = np.maximum(w, 0)
        C = U @ np.diag(w) @ U.T

        # Descale C with D
        C = D @ C @ D

        # Check for convergence
        if norm(C - A, 'fro') < tol:
            break

    return C

N = 100  # Maximum number of iterations
tol = 1e-6  # Tolerance for convergence
start=time.time()
X = nearest_correlation_matrix_diagonal_scaling(A, N, tol)
end=time.time()
print(end-start)
print(X)
print(frobenius_norm(W-X))

0.0240936279296875
[[ 0.95836323  0.02254897  0.00687844 -0.0126005  -0.79703701]
 [ 0.02254897  0.98709111 -0.53836868 -0.36547781 -0.46569515]
 [ 0.00687844 -0.53836868  0.9988129  -0.03709033  0.09035327]
 [-0.0126005  -0.36547781 -0.03709033  0.99600509  0.41223837]
 [-0.79703701 -0.46569515  0.09035327  0.41223837  0.94100121]]
0.11381738385897265


In [ ]:
import numpy as np

def random_matrix_approximation(A, S):
    # Initialize C as a copy of A
    C = np.copy(A)

    for s in range(S):
        n = A.shape[0]
        # Generate a random matrix R with i.i.d. standard normal entries
        R = np.random.normal(0, 1, (n, n))

        # Compute the product of C and R: P = CR
        P = np.dot(C, R)

        # Perform the spectral decomposition of P : P = U ΛU^T
        eig_vals, U = np.linalg.eigh(P)
        Λ = np.diag(eig_vals)

        # Set Λ to a diagonal matrix with its diagonal elements clamped to the interval [0, 1]
        Λ[Λ < 0] = 0
        Λ[Λ > 1] = 1

        # Reconstruct P as P = U ΛU^T
        P = np.dot(U, np.dot(Λ, U.T))


        # Update C using the approximation: C ← C - C(P - I)C
        I = np.eye(n)
        C = C - C @ (P - I) @ C

    # Normalize the diagonal of C to be all ones:
    C = C / np.sqrt(np.diag(C))

    return C



S = 100  # Number of random samples
start=time.time()
X = random_matrix_approximation(A, S)
end=time.time()
print(end-start)
print(X)
print(frobenius_norm(W-X))

<ipython-input-34-1a8a96afa7dc>:29: RuntimeWarning: overflow encountered in matmul
  C = C - C @ (P - I) @ C


LinAlgError: ignored